In [1]:
# Importando bibliotecas
import PyPDF2
import re
import pandas as pd


In [2]:
# Carrega nome das variaveis
tabela_nome_var = pd.read_excel('../dados/auxiliares/tabela_nome_var.xlsx')
nomes = tabela_nome_var['nomes']
# Carrega nome das ufs
ufs = pd.read_excel('../dados/auxiliares/uf.xlsx')


In [5]:
def extrai_texto_uf(uf):
    # Define uf em maiscula
    uf_texto = uf.upper()

    # Le pdf
    reader = PyPDF2.PdfReader('../dados/dados_pdf/matriculas/' + uf + '.pdf')

    # Une texto
    texto = ''
    for i in reader.pages:
        texto += i.extract_text()
        
    # Dividindo o texto em linhas
    texto_int = texto.split('\n')
    # Filtrando linhas que começam com o valor de uf_texto
    texto_int = [linha for linha in texto_int if re.search(f"^{uf_texto}", linha)]
    
    # Removendo linhas que contêm a palavra 'TOTAL'
    texto_int = [linha for linha in texto_int if not re.search('TOTAL', linha)]
    
    # Removendo caracteres não numéricos do início de cada linha
    texto_int = [re.sub("^\\D+", "", linha) for linha in texto_int]
    
    # Substituindo espaços múltiplos por um único espaço
    texto_int = [re.sub(r'\s+', ' ', linha) for linha in texto_int]
    
    # Removendo pontos
    texto_int = [re.sub(r'\.', '', linha) for linha in texto_int]
    
    # Substituindo vírgulas por pontos
    texto_int = [re.sub(r',', '.', linha) for linha in texto_int]
    
    # Removendo problema com alguns string que contém rodapé/cabeçalho
    texto_int = [re.sub(" MATRÍCULAS.+Página [0-9] de [0-9]", "", linha) for linha in texto_int]
    texto_int = [re.sub("[0-9]{2}/.+Página [0-9] de [0-9]", "", linha) for linha in texto_int]
    
    # Dividindo cada linha em palavras
    texto_int = [linha.split() for linha in texto_int]
    
    # Criando um DataFrame a partir do texto_int
    df_matriculas = pd.DataFrame(texto_int, columns = nomes)

    # Retorna dados
    return(df_matriculas)

In [6]:
# Estabele dados base
base = extrai_texto_uf('ac')

# Itera sobre ufs para gerar dados de matriculas
for i in ufs['uf'][1:]:
    base = pd.concat([base, extrai_texto_uf(i)])

In [9]:
# Salva dados de matriculas
base.to_excel('../dados/matriculas.xlsx')